In [1]:
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GATv2Conv,DynamicEdgeConv, EdgeConv,RGCNConv, TransformerConv, GINConv, global_mean_pool
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
import warnings
import sys
sys.path.append('..')
from dataset.data import get_dataset
from utils.table_to_graph import Table2GraphTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
import argparse,os,random
from collections import Counter
from typing import Dict, List, Tuple
# 기존 ArgumentParser로부터 같거나 유사한 파라미터들을 Namespace로 직접 생성
args = argparse.Namespace(
    random_seed=42,
    train_epochs=200,
    batch_size=64,
    input_dim=768,
    hidden_dim=128,
    num_layers=4,
    dropout_rate=0.3,
    threshold=0.5,
    heads=8,
    model='NORM_GNN',
    source_dataset_name='cleveland',
    target_dataset_name='hungarian',
    few_shot=4,
    dataset_seed=4,
    source_lr=0.0001,
    llm_model='gpt2',
    use_gpu=True,
    des=None,  # 실험 메모를 남기는 용도
    baseline=[],  # 'Logistic_Regression' or 'XGBoost' 등을 리스트로 넣을 수 있음
    graph_path="/storage/personal/eungyeop/dataset/graph",
    table_path="/storage/personal/eungyeop/dataset/table",
    model_type='NORM_GNN',
    graph_type='star',
    FD='N',
    label=False  # --label 플래그 사용 시 True
)
import torch
torch.use_deterministic_algorithms(False)


/home/eungyeop/anaconda3/envs/featllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from torch_geometric.nn import GCNConv, GATConv

/home/eungyeop/anaconda3/envs/featllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch_geometric
import torch

In [4]:
print(torch_geometric.__version__)

2.5.2


In [8]:
print(torch.__version__)

2.1.0+cu121


2.1.0+cu121


In [3]:
os.environ['PYTHONHASHSEED'] = str(args.random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.random_seed)
    torch.cuda.manual_seed_all(args.random_seed)

if args.label:
    file_path = os.path.join(args.graph_path, f"{args.graph_type}_{args.FD}_label_{args.target_dataset_name}.pkl")
else:
    file_path = os.path.join(args.graph_path, f"{args.graph_type}_{args.FD}_{args.target_dataset_name}.pkl")



In [4]:
dataset_and_class = {
    "adult" : ['income', ['no','yes']],
    "bank" : ['Class', ['no','yes']],
    "blood" : ['Class',['no','yes']],
    "car" : ['class',['unacceptable','acceptable','good','very good']],
    "communities" : ['ViolentCrimesPerPop',['high','medium','low']],
    "credit-g" : ['class', ['no','yes']],
    "diabetes": ['Outcome',['no','yes']],
    "myocardial" : ['ZSN',['no','yes']],
    "cleveland": ['target_binary', ['no','yes']],
    "hungarian": ['target_binary', ['no','yes']],
    "switzerland": ['target_binary', ['no','yes']],
    "heart_statlog": ['target_binary', ['no','yes']],
    "heart": ['target_binary', ['no','yes']]
}

In [5]:
def preprocessing( DATASETS: pd.DataFrame, data_name: str) -> Tuple[pd.DataFrame, np.ndarray]:
    """데이터셋 전처리"""
    assert data_name in dataset_and_class, f"{data_name} is not a valid dataset name"

    class_name = dataset_and_class[data_name][0]
    class_values = dataset_and_class[data_name][1]
    
    class_mapping = {label: idx for idx, label in enumerate(class_values)}
    
    X = DATASETS.drop(class_name, axis=1)
    X = X.reset_index(drop=True)

    y = DATASETS[class_name]
    y = y.map(class_mapping).astype(int)
    y = y.reset_index(drop=True)
    
    return X, y

In [6]:
base_path = "/storage/personal/eungyeop/dataset/table/"
data_source = "label_table" if args.label else "origin_table"
file_path = os.path.join(base_path, data_source, f"{args.source_dataset_name}.csv")
cleveland = pd.read_csv(file_path)
X, y = preprocessing(cleveland, args.source_dataset_name)
print(X.head())
print(y.head())

    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4
0    0
1    1
2    1
3    0
4    0
Name: target_binary, dtype: int64


In [7]:
print(len(X))
print(len(y))

303
303


In [8]:
for_num = 5

for i in range(for_num -3):
    print(i)

0
1


In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

# gpt2 모델/토크나이저 로드
model = GPT2Model.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def get_mean_embedding(sentence):
    # 토크나이징
    inputs = tokenizer(sentence, return_tensors="pt", add_special_tokens=False)
    # GPT-2 forward (gradient 계산 X)
    with torch.no_grad():
        outputs = model(**inputs)
    # last_hidden_state.shape = [batch_size, seq_len, hidden_size]
    # 시퀀스 길이(dim=1)에 대해 평균 -> (batch_size=1, hidden_size) -> (hidden_size,)로 squeeze
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings

# 테스트 문장
sent1 = "AAAAAAAAAAAAAAAAAA"
sent2 = "I just bought a new phone and I'm really excited about it."

# 임베딩 및 코사인 유사도 계산
emb1 = get_mean_embedding(sent1)
emb2 = get_mean_embedding(sent2)
cos_sim = torch.nn.functional.cosine_similarity(emb1, emb2, dim=0)

print("Sentence 1:", sent1)
print("Sentence 2:", sent2)
print("Cosine similarity:", cos_sim.item())


RuntimeError: The size of tensor a (3) must match the size of tensor b (14) at non-singleton dimension 1

In [22]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Sentence-BERT 모델 로드
# (가장 많이 쓰이는 'all-MiniLM-L6-v2'를 예시로 사용)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 예시 문장
sent1 = " Individuals with a family history of diabetes are more likely to inherit genes that increase their risk of developing the condition."
sent2 = "Consuming excessive calories and engaging in little to no physical exercise can lead to weight gain, which reduces insulin sensitivity and significantly heightens the chance of developing diabetes"
sent3 = "race"
# 문장 임베딩 구하기
embedding1 = model.encode(sent1, convert_to_tensor=True)
embedding2 = model.encode(sent2, convert_to_tensor=True)
embedding3 = model.encode(sent3, convert_to_tensor=True)

# 코사인 유사도 계산 (문장1 vs 문장2)
cos_sim = util.cos_sim(embedding1, embedding2)
print("Sentence 1:", sent1)
print("Sentence 2:", sent2)
print("Cosine similarity:", cos_sim.item())
print()
cos_sim2 = util.cos_sim(embedding1, embedding3)
print("Sentence 1:", sent1)
print("Sentence 2:", sent3)
print("Cosine similarity:", cos_sim2.item())
print()
cos_sim3 = util.cos_sim(embedding2, embedding3)
print("Sentence 1:", sent2)
print("Sentence 2:", sent3)
print("Cosine similarity:", cos_sim3.item())


Sentence 1:  Individuals with a family history of diabetes are more likely to inherit genes that increase their risk of developing the condition.
Sentence 2: Consuming excessive calories and engaging in little to no physical exercise can lead to weight gain, which reduces insulin sensitivity and significantly heightens the chance of developing diabetes
Cosine similarity: 0.4125227928161621

Sentence 1:  Individuals with a family history of diabetes are more likely to inherit genes that increase their risk of developing the condition.
Sentence 2: race
Cosine similarity: 0.09437273442745209

Sentence 1: Consuming excessive calories and engaging in little to no physical exercise can lead to weight gain, which reduces insulin sensitivity and significantly heightens the chance of developing diabetes
Sentence 2: race
Cosine similarity: 0.009398882277309895


In [24]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Sentence-BERT 모델 로드
# (가장 많이 쓰이는 'all-MiniLM-L6-v2'를 예시로 사용)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 예시 문장
sent1 = "'the general type of occupation of an individual"
sent2 = "marital status of an individual"
sent3 = "race"
# 문장 임베딩 구하기
embedding1 = model.encode(sent1, convert_to_tensor=True)
embedding2 = model.encode(sent2, convert_to_tensor=True)
embedding3 = model.encode(sent3, convert_to_tensor=True)

# 코사인 유사도 계산 (문장1 vs 문장2)
cos_sim = util.cos_sim(embedding1, embedding2)
print("Sentence 1:", sent1)
print("Sentence 2:", sent2)
print("Cosine similarity:", cos_sim.item())
print()



Sentence 1: 'the general type of occupation of an individual
Sentence 2: marital status of an individual
Cosine similarity: 0.3343375325202942



In [25]:
import torch
from sentence_transformers import SentenceTransformer, util

# 예시 딕셔너리
desc_dict = {
    'workclass': 'a general term to represent the employment status of an individual',
    'education': 'the highest level of education achieved by an individual',
    'marital-status': 'marital status of an individual',
    'occupation': 'the general type of occupation of an individual',
    'relationship': 'what this individual is relative to others',
    'race': 'race',
    'gender': 'gender',
    'native-country': 'country of origin for an individual'
}

# Sentence-BERT 모델 로드
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 1) 각 description을 임베딩하고, 리스트에 모읍니다.
desc_embeddings = []
for key, desc in desc_dict.items():
    embedding = model.encode(desc, convert_to_tensor=True)  # shape: [embedding_dim]
    desc_embeddings.append(embedding)

# 2) 리스트에 담긴 임베딩들을 하나의 텐서로 만듭니다.
#    - len(desc_embeddings) = N, each embedding shape = [embedding_dim]
#    - 최종 shape = [N, embedding_dim]
desc_embeddings = torch.stack(desc_embeddings, dim=0)

print("desc_embeddings.shape:", desc_embeddings.shape)

# 3) 임베딩들 간 내적(dot product)을 계산합니다 (pairwise dot product).
#    - 결과 shape = [N, N]
dot_products = desc_embeddings @ desc_embeddings.T

# 4) 결과 출력
print("Pairwise dot product matrix:")
print(dot_products)


desc_embeddings.shape: torch.Size([8, 384])
Pairwise dot product matrix:
tensor([[1.0000, 0.2635, 0.4289, 0.6031, 0.2788, 0.1121, 0.1680, 0.1824],
        [0.2635, 1.0000, 0.2064, 0.3038, 0.2933, 0.1259, 0.2208, 0.1600],
        [0.4289, 0.2064, 1.0000, 0.3502, 0.4081, 0.1487, 0.2543, 0.2180],
        [0.6031, 0.3038, 0.3502, 1.0000, 0.4382, 0.1711, 0.2949, 0.2623],
        [0.2788, 0.2933, 0.4081, 0.4382, 1.0000, 0.2541, 0.2973, 0.3390],
        [0.1121, 0.1259, 0.1487, 0.1711, 0.2541, 1.0000, 0.3674, 0.2123],
        [0.1680, 0.2208, 0.2543, 0.2949, 0.2973, 0.3674, 1.0000, 0.1916],
        [0.1824, 0.1600, 0.2180, 0.2623, 0.3390, 0.2123, 0.1916, 1.0000]],
       device='cuda:0')


In [33]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# 1) 당뇨 관련 변수를 가정하고, 각 변수의 설명을 딕셔너리에 저장
desc_dict = {
    "BMI": "A numerical index calculated from height and weight",
    "BloodSugar": "The concentration of glucose in a person's blood",
    "EyeColor": "The color or shade of a person's irises",
    "FamilyHistory": "Any relevant conditions or traits observed among an individual's relatives",
    "PhysicalActivity": "The frequency and intensity of exercise or movement performed regularly",
    "Age": "A measure, in years, of how long someone has lived",
    "Hypertension": "A state of having blood pressure above a typical or normal range",
    "DietType": "An overall pattern of food intake or eating habits",
    "Cholesterol": "A type of lipid found in the blood, generally classified as HDL or LDL",
    "AlcoholIntake": "The frequency and amount of alcoholic beverages consumed"
}

# 2) Hugging Face 모델 로드
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# 3) 임베딩 추출 함수 정의
def get_embedding(text):
    # 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # 모델 실행
    with torch.no_grad():
        outputs = model(**inputs)
        
    # [CLS] 토큰 임베딩 추출 (문장 표현)
    sentence_embedding = outputs.last_hidden_state[:, 0, :]
    # 정규화
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    return sentence_embedding
# 4) 각 description 임베딩 계산 후 리스트에 저장
desc_embeddings = []
keys_list = list(desc_dict.keys())

for key in keys_list:
    desc = desc_dict[key]
    embedding = get_embedding(desc)
    desc_embeddings.append(embedding)

# 5) 임베딩들을 하나의 텐서로 스택 -> [N, embedding_dim]
desc_embeddings = torch.cat(desc_embeddings, dim=0)

# 6) 임베딩 쌍 간 코사인 유사도 계산 -> [N, N] 행렬
cos_sim_matrix = torch.mm(desc_embeddings, desc_embeddings.transpose(0, 1))

# 7) 결과 출력
print("Variables:", keys_list)
print("\nPairwise Cosine Similarity Matrix (rows/cols correspond to variables above):")
print(cos_sim_matrix)

Variables: ['BMI', 'BloodSugar', 'EyeColor', 'FamilyHistory', 'PhysicalActivity', 'Age', 'Hypertension', 'DietType', 'Cholesterol', 'AlcoholIntake']

Pairwise Cosine Similarity Matrix (rows/cols correspond to variables above):
tensor([[1.0000, 0.8524, 0.8144, 0.7919, 0.7575, 0.8467, 0.8314, 0.8009, 0.8473,
         0.8331],
        [0.8524, 1.0000, 0.8078, 0.8399, 0.7781, 0.8499, 0.8535, 0.7361, 0.8417,
         0.8722],
        [0.8144, 0.8078, 1.0000, 0.8036, 0.7741, 0.8255, 0.8175, 0.7977, 0.8477,
         0.7883],
        [0.7919, 0.8399, 0.8036, 1.0000, 0.7553, 0.8674, 0.8315, 0.8089, 0.8454,
         0.8235],
        [0.7575, 0.7781, 0.7741, 0.7553, 1.0000, 0.7864, 0.7952, 0.7907, 0.7465,
         0.8532],
        [0.8467, 0.8499, 0.8255, 0.8674, 0.7864, 1.0000, 0.8714, 0.8218, 0.8486,
         0.8586],
        [0.8314, 0.8535, 0.8175, 0.8315, 0.7952, 0.8714, 1.0000, 0.8371, 0.8632,
         0.8085],
        [0.8009, 0.7361, 0.7977, 0.8089, 0.7907, 0.8218, 0.8371, 1.0000, 0.8286,

In [4]:
import transformers 
import torch

model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

pipeline = transformers.pipeline( "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto", )

messages = [ {"role": "system", "content": "You are an expert trained on healthcare and biomedical domain!"}, {"role": "user", "content": "I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, increased sensitivity to cold, and dry, itchy skin. What is the diagnosis here?"}, ]

prompt = pipeline.tokenizer.apply_chat_template( messages, tokenize=False, add_generation_prompt=True )

terminators = [ pipeline.tokenizer.eos_token_id, pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>") ]

outputs = pipeline(prompt, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9, ) 
print(outputs[0]["generated_text"][len(prompt):])


/home/eungyeop/anaconda3/envs/featllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like ContactDoctor/Bio-Medical-Llama-3-8B is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [4]:
import spacy
import scispacy
nlp = spacy.load("en_ner_bionlp13cg_md")

doc = nlp("blood pressure")
for ent in doc.ents:
    print(ent.text, ent._.umls_ents)

OSError: [E050] Can't find model 'en_ner_bionlp13cg_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
import spacy

nlp = spacy.load("en_ner_bionlp13cg_md")

text = "The patient has high blood pressure and elevated glucose levels."
doc = nlp(text)

for ent in doc.ents:
    print(f"Text: {ent.text}")
    print(f"UMLS Concepts: {ent._.umls_ents}")  # List of (CUI, score)


/home/eungyeop/anaconda3/envs/featllm/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bionlp13cg_md' (0.5.0) was trained with spaCy v3.2.3 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Text: patient


AttributeError: [E046] Can't retrieve unregistered extension attribute 'umls_ents'. Did you forget to call the `set_extension` method?